# Húsnæðislánareiknivél

Þetta er lauflétt húsnæðislánareiknivél sem ég bjó til til þess að skilja betur hvernig húsnæðislán virka.

Áður en þú byrjar að nota reiknivélina þarftu að setja upp þau forritasöfn sem eru nauðsynleg. Það er gert með því að keyra `python -m pip install -r requirements.txt`. Þessi skrifta hefur verið prófuð á Python útgáfu 3.11.2.

Til að nota reiknivélina er nóg breyta breytunum í næsta hólfi (Stillingar) og keyra reiknivélina (Run all).

## Verkefnalisti:

- [ ] Yfirlit yfir tölfræði lána (t.d. heildarkostnað, lægsta, hæsta og miðgildi afborgunar)
- [ ] Gröf og yfirlit yfir eignamyndun

In [ ]:
# Stillingar - Hægt að breyta þessum gildum

# Sameiginlegar stillingar fyrir öll lán
lansupphaed: int = 20000000

# Óverðtryggð lán
lanstimi_overdtryggt: int = 40
vextir_overdtryggt: float = 0.0925  # Ársvextir

# Verðtryggð lán
verdbolga: float = 0.0988  # Áætluð árleg
lanstimi_verdtryggt: int = 15
vextir_verdtryggt: float = 0.0265  # Ársvextir

# Fasteign
verd: int = 60000000
fasteignamat: int = 50000000

# Mánaðarleg viðbótargreiðsla inn á lán
vidbotargreidsla: int = 50000

In [ ]:
# Afleiddar breytur - Ekki breyta

fjoldi_afborgana_overdtryggt = 12 * lanstimi_overdtryggt
fjoldi_afborgana_verdtryggt = 12 * lanstimi_verdtryggt

In [ ]:
# Innfluttningur á kóðasöfnum

from typing import Generator

import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Reikniföll

def jafnar_afborganir(
    lansupphaed: int,
    fjoldi_afborgana: int,
    vextir: float,
    vidbotargreidsla: int,
) -> Generator[tuple[int, int, int, int, int, int, int], None, None]:
    '''
    Þetta fall reiknar út greiðslur lán með jöfnum afborgunum.

    Fallið skilar generator sem skilar við hverja ítrun færslu sem inniheldur:
        - númer mánaðar
        - greiðslu af höfuðstól
        - vaxtagreiðsla
        - viðbótargreiðsla inn á höfuðstól
        - heildar greiðsla
        - verðbætur (Alltaf 0 þar sem verðið er óverðtryggt)
        - stöðu höfuðstóls
    '''
    hofudstoll = float(lansupphaed)
    for i in range(fjoldi_afborgana):
        afborgun_af_hofudstol = hofudstoll / (fjoldi_afborgana - i + 1) + vidbotargreidsla
        vaxtagreidsla = hofudstoll * (vextir) / 12
        
        if hofudstoll - afborgun_af_hofudstol <= 0:
            afborgun_af_hofudstol = hofudstoll

        heildargreidsla = afborgun_af_hofudstol + vaxtagreidsla

        yield (
            i+1,
            int(round(afborgun_af_hofudstol, 0)),
            int(round(vaxtagreidsla, 0)),
            vidbotargreidsla,
            int(round(heildargreidsla, 0)),
            0,
            int(round(hofudstoll, 0)),
        )

        hofudstoll -= afborgun_af_hofudstol 


def jafnar_afborganir_verdtryggt(
    lansupphaed: int,
    fjoldi_afborgana: int,
    vextir: float,
    verdbolga: float,
    vidbotargreidsla: int,
) -> Generator[tuple[int, int, int, int, int, int, int], None, None]:
    '''
    Þetta fall reiknar út greiðslur lán með jöfnum afborgunum.

    Fallið skilar generator sem skilar við hverja ítrun færslu sem inniheldur:
        - númer mánaðar
        - greiðslu af höfuðstól
        - vaxtagreiðsla
        - viðbótargreiðsla inn á höfuðstól
        - heildar greiðsla
        - verðbætur
        - stöðu höfuðstóls
    '''
    # Verðbólgan er gefin upp á ársgrundvelli. Við verðum 
    # því að umreikna hana yfir á mánaðargrundvöll.
    # Við þurfum að leysa eftirfarandi jöfnu:
    #   1 + V = (1 + m)^12
    # þar sem V er árleg verðbólga og m er verðbólgan á mánaðargurndvelli.
    # Leysum hana og fáum
    #   m = (1 + V)^(1/12) - 1
    manadarleg_verdbolga = (1 + verdbolga)**(1/12) - 1

    hofudstoll = float(lansupphaed)
    for i in range(fjoldi_afborgana):
        verdbaetur =  hofudstoll * manadarleg_verdbolga
        hofudstoll += verdbaetur

        vaxtagreidsla = hofudstoll * (vextir / 12)
        afborgun_af_hofudstol = hofudstoll / (fjoldi_afborgana - i ) + vidbotargreidsla  # afborgun_af_hofudstol * (1 + ((verdbolga / 100) / 12))
        
        if hofudstoll - afborgun_af_hofudstol <= 0:
            afborgun_af_hofudstol = hofudstoll
        
        heildargreidsla = afborgun_af_hofudstol + vaxtagreidsla

        yield (
            i+1,
            int(round(afborgun_af_hofudstol, 0)),
            int(round(vaxtagreidsla, 0)),
            vidbotargreidsla,
            int(round(heildargreidsla, 0)),
            int(round(verdbaetur, 0)),
            int(round(hofudstoll, 0)),
        )

        hofudstoll -= afborgun_af_hofudstol


def jafnar_greidslur(
    lansupphaed: int,
    fjoldi_afborgana: int,
    vextir: float,
    vidbotargreidsla: int,
) -> Generator[tuple[int, int, int, int, int, int, int], None, None]:
    '''
    Þetta fall reiknar út greiðslur láns með jöfnum greiðslum.

    Greiðslur inn á höfuðstól eru reiknaðar með formúlunni:

        M = H[r(1 + r)^n]/[(1 + r)^n – 1]

    þar sem:
        - M er mánaðarleg heildargreiðsla
        - H er höfuðstóll láns
        - r er vextir á mánuði (Vextir á ársgrundvelli / 12)
        - n er fjöldi afborgana (mánuðir)

    Fallið skilar generator sem skilar við hverja ítrun færslu sem inniheldur:
        - númer mánaðar
        - greiðslu af höfuðstól
        - vaxtagreiðsla
        - viðbótargreiðsla
        - heildar greiðsla
        - verðbætur (Alltaf 0 þar sem verðið er óverðtryggt)
        - stöðu höfuðstóls
    '''
    hofudstoll = float(lansupphaed)

    r = vextir / 12

    for i in range(fjoldi_afborgana):
        vaxtagreidsla = hofudstoll * r
        heildargreidsla = hofudstoll * (r * (1 + r)**(fjoldi_afborgana - i)) / ((1 + r)**(fjoldi_afborgana - i) - 1) + vidbotargreidsla
        afborgun_af_hofudstol = heildargreidsla - vaxtagreidsla

        if hofudstoll - afborgun_af_hofudstol <= 0:
            afborgun_af_hofudstol = hofudstoll

        yield (
            i+1,
            int(round(afborgun_af_hofudstol, 0)),
            int(round(vaxtagreidsla, 0)),
            vidbotargreidsla,
            int(round(heildargreidsla, 0)),
            0,
            int(round(hofudstoll, 0)),
        )

        hofudstoll -= afborgun_af_hofudstol


def jafnar_greidslur_verdtryggt(
    lansupphaed: int,
    fjoldi_afborgana: int,
    vextir: float,
    verdbolga: float,
    vidbotargreidsla: int,
) -> Generator[tuple[int, int, int, int, int, int], None, None]:
    '''
    Þetta fall reiknar út greiðslur verðtryggðs láns með jöfnum greiðslum.

    Greiðslur inn á höfuðstól eru reiknaðar með formúlunni:

        M = H[r(1 + r)^n]/[(1 + r)^n – 1]

    þar sem:
        - M er mánaðarleg heildargreiðsla
        - H er höfuðstóll láns
        - r er vextir á mánuði (Vextir á ársgrundvelli / 12)
        - n er fjöldi afborgana (mánuðir)

    Þar sem lánið er verðtryggt þarf að endurreikna greiðsluna inn á höfuðstól
    í hverjum mánuði miðað við þann fjölda afborgana sem eftir eru af láninu.

    Fallið skilar generator sem skilar við hverja ítrun færslu sem inniheldur:
        - númer mánaðar
        - greiðslu af höfuðstól
        - vaxtagreiðsla
        - viðbótargrieðsla
        - heildar greiðsla
        - verðbætur
        - stöðu höfuðstóls
    '''
    hofudstoll = float(lansupphaed)
    r = vextir / 12

    # Verðbólgan er gefin upp á ársgrundvelli. Við verðum 
    # því að umreikna hana yfir á mánaðargrundvöll.
    # Við þurfum að leysa eftirfarandi jöfnu:
    #   1 + V = (1 + m)^12
    # þar sem V er árleg verðbólga og m er verðbólgan á mánaðargurndvelli.
    # Leysum hana og fáum
    #   m = (1 + V)^(1/12) - 1
    manadarleg_verdbolga = (1 + verdbolga)**(1/12) - 1

    for i in range(fjoldi_afborgana):
        verdbaetur =  hofudstoll * manadarleg_verdbolga
        hofudstoll += verdbaetur

        fjoldi_afborgana_eftir = fjoldi_afborgana - i
        heildargreidsla = hofudstoll * (r * (1 + r)**fjoldi_afborgana_eftir) / ((1 + r)**fjoldi_afborgana_eftir - 1) + vidbotargreidsla
        vaxtagreidsla = hofudstoll * r
        afborgun_af_hofudstol = heildargreidsla - vaxtagreidsla

        yield (
            i,
            int(round(afborgun_af_hofudstol, 0)),
            int(round(vaxtagreidsla, 0)),
            vidbotargreidsla,
            int(round(heildargreidsla, 0)),
            int(round(verdbaetur, 0)),
            int(round(hofudstoll, 0)),
        )

        hofudstoll -= afborgun_af_hofudstol

In [ ]:
# Útreikningur

jafnar_afborganir_overdtryggt = pd.DataFrame(
    jafnar_afborganir(
        lansupphaed=lansupphaed,
        fjoldi_afborgana=fjoldi_afborgana_overdtryggt,
        vextir=vextir_overdtryggt,
        vidbotargreidsla=vidbotargreidsla,
    ),
    columns=[
        'Mánuður',
        'Afborgun',
        'Vaxtagreiðsla',
        'Viðbótargreiðsla',
        'Heildargreiðsla',
        'Verðbætur',
        'Höfuðstóll',
    ],
).set_index('Mánuður')

plt.plot(jafnar_afborganir_overdtryggt['Afborgun'], label='Afborgun')
plt.plot(jafnar_afborganir_overdtryggt['Vaxtagreiðsla'], label='Vaxtagreiðsla')
plt.plot(jafnar_afborganir_overdtryggt['Heildargreiðsla'], label='Heildargreiðsla')

plt.xlabel('Mánuður')
plt.ylabel('Upphæð (kr.)')
plt.title('Greiðslubyrði láns með jöfnum afborgunum')
plt.legend()


jafnar_greidslur_overdtryggt = pd.DataFrame(
    jafnar_greidslur(
        lansupphaed=lansupphaed,
        fjoldi_afborgana=fjoldi_afborgana_overdtryggt,
        vextir=vextir_overdtryggt,
        vidbotargreidsla=vidbotargreidsla,
    ),
    columns=[
        'Mánuður',
        'Afborgun',
        'Vaxtagreiðsla',
        'Viðbótargreiðsla',
        'Heildargreiðsla',
        'Verðbætur',
        'Höfuðstóll',
    ],
).set_index('Mánuður')

plt.figure()
plt.plot(jafnar_greidslur_overdtryggt['Afborgun'], label='Afborgun')
plt.plot(jafnar_greidslur_overdtryggt['Vaxtagreiðsla'], label='Vaxtagreiðsla')
plt.plot(jafnar_greidslur_overdtryggt['Heildargreiðsla'], label='Heildargreiðsla')

plt.xlabel('Mánuður')
plt.ylabel('Upphæð (kr.)')
plt.title('Greiðslubyrði láns með jöfnum greiðslum')
plt.legend()


jafnar_afborganir_verdtryggt = pd.DataFrame(
    jafnar_afborganir_verdtryggt(
        lansupphaed=lansupphaed,
        fjoldi_afborgana=fjoldi_afborgana_verdtryggt,
        vextir=vextir_verdtryggt,
        verdbolga=verdbolga,
        vidbotargreidsla=vidbotargreidsla,
    ),
    columns=[
        'Mánuður',
        'Afborgun',
        'Vaxtagreiðsla',
        'Viðbótargreiðsla',
        'Heildargreiðsla',
        'Verðbætur',
        'Höfuðstóll',
    ],
).set_index('Mánuður')

plt.figure()
plt.plot(jafnar_afborganir_verdtryggt['Afborgun'], label='Afborgun')
plt.plot(jafnar_afborganir_verdtryggt['Vaxtagreiðsla'], label='Vaxtagreiðsla')
plt.plot(jafnar_afborganir_verdtryggt['Heildargreiðsla'], label='Heildargreiðsla')

plt.xlabel('Mánuður')
plt.ylabel('Upphæð (kr.)')
plt.title('Greiðslubyrði verðtryggðs láns með jöfnum afborgunum')
plt.legend()

jafnar_greidslur_verdtryggt = pd.DataFrame(
    jafnar_greidslur_verdtryggt(
        lansupphaed=lansupphaed,
        fjoldi_afborgana=fjoldi_afborgana_verdtryggt,
        vextir=vextir_verdtryggt,
        verdbolga=verdbolga,
        vidbotargreidsla=vidbotargreidsla,
    ),
    columns=[
        'Mánuður',
        'Afborgun',
        'Vaxtagreiðsla',
        'Viðbótargreiðsla',
        'Heildargreiðsla',
        'Verðbætur',
        'Höfuðstóll',
    ],
).set_index('Mánuður')

plt.figure()
plt.plot(jafnar_greidslur_verdtryggt['Afborgun'], label='Afborgun')
plt.plot(jafnar_greidslur_verdtryggt['Vaxtagreiðsla'], label='Vaxtagreiðsla')
plt.plot(jafnar_greidslur_verdtryggt['Heildargreiðsla'], label='Heildargreiðsla')

plt.xlabel('Mánuður')
plt.ylabel('Upphæð (kr.)')
plt.title('Greiðslubyrði verðtryggðs láns með jöfnum greiðslum')
plt.legend()